In [1]:
import torch
import torch.nn.functional as F
import json
import numpy as np
from espnet.utils.io_utils import LoadInputsAndTargets
from espnet.utils.training.batchfy import make_batchset
from espnet.asr.pytorch_backend.asr_init import load_trained_model
from espnet.asr.pytorch_backend.asr import CustomConverter
import cmudict
import re
import Hmax
def getPsuedoKeyword(target_phone, already_present):
    a = cmudict.dict()
    b = cmudict.words()
    found = False
    for word in b:
        for lst in a[word]:
            for phone in lst:
                if(target_phone == phone and already_present.get(word) == None):
                    already_present[word] = 1
                    return word
                    
                if(re.search(target_phone,phone) and len(target_phone) !=1 and already_present.get(word) == None):
                    return word

device = torch.device("cuda")

with open("/home/ram/chaitanay/features/Great/deltafalse/data.json", "r") as f:
    train_json = json.load(f)["utts"]

# with open("/root/kws_data/dump/test/deltafalse/data.json", "r") as f:
#     test_json = json.load(f)["utts"]

# with open("/root/kws_data/dump/dev/deltafalse/data.json", "r") as f:
#     dev_json = json.load(f)["utts"]

train_data_batches = make_batchset(train_json, 1)
# test_data_batches = make_batchset(test_json, 1)
# dev_data_batches = make_batchset(dev_json, 1)

load_tr = LoadInputsAndTargets(
        mode='asr', load_output=True, preprocess_conf=None,
        preprocess_args={'train': True}  # Switch the mode of preprocessing
)

converter = CustomConverter(subsampling_factor=1, dtype=torch.float32)

model, train_args = load_trained_model("/home/ram/chaitanay/model/model.acc.best")
model = model.to(device=device)
model.dec.sampling_probability = 1.0

phone_to_int = dict(zip(train_args.char_list, np.arange(len(train_args.char_list))))
keyword = "G R EY T"
keyword_tokens = torch.tensor([[phone_to_int[phn] for phn in keyword.split(" ")]]).to(device)

encoder_output = 0

# def get_att_score2(att_w_list):
#     atts = [ele.detach().cpu().numpy().flatten() for ele in att_w_list]
#     atts = np.array(atts)
#     sum_att = np.prod(atts, axis=0)
#     att_score = np.trapz(sum_att)
#     return att_score

# def get_att_score(att_w_list):
#     atts = [ele.detach().cpu().numpy().flatten() for ele in att_w_list]
#     atts = np.array(atts)
#     init_att = atts[0]
#     for att in atts[1:]:
#         init_att = np.abs(init_att - att)
#     att_score = np.mean(init_att)
#     return att_score
encoder_output = 0
# with open("train_confidences_ctc_att_small_sp1_as.txt", "a") as f:
for row in train_data_batches:
        data_input = [load_tr(row)]
        data = converter(data_input, device)
        encoder_output, hlens, _ = model.enc(data[0], data[1])
        break
        
encoder_output = encoder_output.detach().cpu().numpy()

        
        
Q_value = dict()
column =0
pickle_in = open("Confusion.Matrix","rb")
Confusion_Matrix = pickle.load(pickle_in)
pickle_in = open("Deletion.vector","rb")
Deletion_vector = pickle.load(pickle_in)
pickle_in = open("Character_dict","rb")
char_dict = pickle.load(pickle_in)
integer_dict = dict(map(reverse,char_dict.items()))

for ph in keyword:
    phone = ph.upper()
    print("calculating for phone ",phone)
    Q_x = []
    number = 0
    word_dict = dict()
    for n in range(10):
        class_Hm = Hmax()
        psd_word = getPsuedoKeyword(phone,word_dict)
        Hmax,op_list=class_Hm.getHmax(psd_word,Encoder_output,0,0,1.0,[])
        for op in op_list:
            oper = op[0]
            src_pos = op[1]
            dst_pos = op[2]
            if(oper == 'replace' and psd_word[dst_pos] = phone ):
                val = Encoder_output[char_dict[phone]][column]*Confusion_Matrix[char_dict[phone]][Hmax[dest_pos]]
                Q_x.append(val)
                number = number + 1
            elif(oper == 'delete' and psd_word[dst_pos] = phone ):
                val = Deletion_vector[phone]
                Q_x.append(val)
                number =number + 1
    sm=0 
    for val in Q_x:
        sm = sm + val
        
    if(Q_value.get(phone) == None):
        Q_value[phone] = sm/float(number)
    else:
        Q_value[phone] = (Q_value[phone] + sm/float(number))/2.0
        

print(Q_value)
        
        
        
        
        #hs_pad is the output of the encoder
#                 ctc_loss = model.ctc(hs_pad, hlens, keyword_tokens)
#                 ctc_loss = float(ctc_loss.cpu().detach().numpy())
#                 att_loss, acc, _, att_w_list = model.dec(hs_pad, hlens, keyword_tokens)
#                 att_score = get_att_score(att_w_list)

#                 att_loss = float(att_loss.cpu().detach().numpy())
#                 f.write(str(ctc_loss) + "," + str(att_loss) + "," + str(acc) + "," + str(att_score) + "," + str(row[0][1]["output"][0]["token"]))
#                 f.write("\n")
                
# with open("test_confidences_ctc_att_small_sp1_as.txt", "a") as f:
#         for row in test_data_batches:
#                 data_input = [load_tr(row)]
#                 data = converter(data_input, device)
#                 hs_pad, hlens, _ = model.enc(data[0], data[1])
#                 ctc_loss = model.ctc(hs_pad, hlens, keyword_tokens)
#                 ctc_loss = float(ctc_loss.cpu().detach().numpy())
#                 att_loss, acc, _, att_w_list = model.dec(hs_pad, hlens, keyword_tokens)
#                 att_score = get_att_score(att_w_list)

#                 att_loss = float(att_loss.cpu().detach().numpy())
#                 f.write(str(ctc_loss) + "," + str(att_loss) + "," + str(acc) + "," + str(att_score) + "," + str(row[0][1]["output"][0]["token"]))
#                 f.write("\n")

# with open("dev_confidences_ctc_att_small_sp1_as.txt", "a") as f:
#         for row in dev_data_batches:
#                 data_input = [load_tr(row)]
#                 data = converter(data_input, device)
#                 hs_pad, hlens, _ = model.enc(data[0], data[1])
#                 ctc_loss = model.ctc(hs_pad, hlens, keyword_tokens)
#                 ctc_loss = float(ctc_loss.cpu().detach().numpy())
#                 att_loss, acc, _, att_w_list = model.dec(hs_pad, hlens, keyword_tokens)
#                 att_score = get_att_score(att_w_list)

#                 att_loss = float(att_loss.cpu().detach().numpy())
#                 f.write(str(ctc_loss) + "," + str(att_loss) + "," + str(acc) + "," + str(att_score) + "," + str(row[0][1]["output"][0]["token"]))
                
                
#encoder output is in encoder_output




ImportError: No module named builtins